# Introducción a Python para IA.

<a href="https://github.com/luiggix/intro_MeIA_2023">intro_MeIA_2023</a> by Luis M. de la Cruz Salas is licensed under <a href="https://creativecommons.org/licenses/by-nc-nd/4.0?ref=chooser-v1">Attribution-NonCommercial-NoDerivatives 4.0 International</a>.

# Objetivos.
Realizar un análisis de datos obtenidos de un sitio web (*Web Scrapping*).

# Web Scrapping

Para obtener los datos de un sitio web, vamos a usar las siguiente bibliotecas:

* [request](https://pypi.org/project/requests/) : permite realizar solicitudes de información a sitios web de una forma muy simple y directa.

* [Beautiful Soup](https://pypi.org/project/beautifulsoup4/) : es una biblioteca para analizar documentos HTML y extraer información requerida para realizar un análisis detallado.

Para este ejercicio, vamos a trabajar con el siguiente sitio web: http://www.ssn.unam.mx/sismicidad/estadisticas/ . Antes de comenzar inspeccionemos el sitio web usando la opción *Inspect* en el sitio web indicado.

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Haciendo el requerimiento a un sitio web.

In [ ]:
URL = "http://www.ssn.unam.mx/sismicidad/estadisticas/"
page = requests.get(URL)

El código anterior se está realizando una solicitud de la información contenida en la URL indicada. La idea es obtener los datos en formato HTML de la URL; y almacenarlos en un objeto de Python.

In [ ]:
print(page, type(page))

### Atributo `.text`

Para ver la información, podemos usar el atributo .`text` del objeto `page`:

In [ ]:
print(type(page.text))# Primero vemos de que tipo es el objeto generado por .text

In [ ]:
print(page.text) # ahora imprimimos el contenido


### Atributo `.content`

Aunque se podría trabajar con el resultado que proporciona `.text` es mejor usar `.content`. Este último atributo proporciona la información en *bytes* crudos que pueden ser decodificados de una mejor manera que la representación en texto. 

In [ ]:
print(type(page.content))

In [ ]:
print(page.content)

## Extrayendo la información con `BeautifulSoup`

Ahora usemos la biblioteca `BeautifulSoup` para extraer la información necesaria para su posterior análisis. Creamos entonces el objeto `soup` como sigue

In [ ]:
soup = BeautifulSoup(page.content, "html.parser")
print(type(soup))

In [ ]:
print(soup)

### Usando la función `prettify()`

In [ ]:
print(soup.prettify())

## Analizando la información.

### Función `find_all()`

In [ ]:
# Buscamos los elementos de tipo div con el siguiente texto "table_header"
año_total = soup.find_all("div", class_="table_header")
año_total

In [ ]:
print(type(año_total), len(año_total))

### Atributo `contents`

In [ ]:
print(año_total[0], type(año_total[0]), año_total[0].contents)
print(año_total[1], type(año_total[1]), año_total[1].contents)

In [ ]:
print(len(año_total[0].contents), año_total[0].contents[0])
print(len(año_total[1].contents), año_total[1].contents[0])

### Generando las columnas

In [ ]:
columnas = [año_total[1].contents[0]]
columnas

Ahora buscamos las columnas restantes con información relevante.

In [ ]:
magnitudes = soup.find_all("div", class_="table_magnitud")
magnitudes

In [ ]:
print(type(magnitudes), len(magnitudes))

In [ ]:
print(type(magnitudes[0]), len(magnitudes[0]), magnitudes[0].contents)

In [ ]:
for m in magnitudes:
    print(type(m), len(m), m.contents)

In [ ]:
for m in magnitudes:
    columnas.append(m.contents[0])

In [ ]:
columnas

### Obteniendo los datos numéricos

Los datos están en un contender de tipo "\<div class="table_item">", entonces para encontrar esa información hacemos lo siguiente:

In [ ]:
data = soup.find_all("div", class_="table_item")
print(type(data), len(data))

In [ ]:
data

Checamos la información antes de pasarla a un DataFrame:

In [ ]:
for r in data:
    print(r.contents[0])
    print(r, type(r), r.contents[0])

### Creando un DataFrame

Ya que se tienen definidas las columnas, podemos definir un DataFrame vacío como sigue

In [ ]:
sismicidad = pd.DataFrame(columns = columnas)
sismicidad

Como se observa, el DataFrame no contiene datos. 

Ahora vamos a recuperar la información que está almacenada en el objeto `data` para agregarla al DataFrame. Recordemos el tipo y longitud del objeto:

In [ ]:
print(type(data), len(data))

Tenemos 1 objeto de tipo `bs4.element.ResultSet` que contiene 340 objetos. 

Cada uno de estos 340 objetos tiene la siguiente información:

In [ ]:
print(data[0], type(data[0]), len(data[0]))

Como puede observarse, los objetos son de tipo `bs4.element.Tag`.

Veamos los primeros 10 de estos objetos:

In [ ]:
data[0:10]

Hacemos uso del atributo `contents` para extraer la información:

In [ ]:
print(data[0].contents, type(data[0].contents), len(data[0].contents))

El contenido es una lista, entonces: 

In [ ]:
print(data[0].contents[0], type(data[0].contents[0]), len(data[0].contents[0]))

Observa que el dato 1990 es de tipo `bs4.element.NavigableString`, pero requerimos que tenga un tipo numérico, en este caso `int` entonces lo vamos a transformar cuando se incluya en el DataFrame.

Usando este aprendizaje, podemos ver toda la información mediante un par de ciclos:

In [ ]:
for i in range(0,340,10): # Salto de 10 en 10
    for r in data[i:i+10]: # Recorrido de los 10 objetos
        print(int(r.contents[0]), end = ' ') # Transformamos cada dato en int
    print()

Vamos a usar el año como índice del DataFrame. Además, debemos relacionar cada dato con su columna.

Usamos la función `zip()` para crear una relación entre el dato numérico y la columna a la que pertenece:

In [ ]:
for i in range(0,10,10):
    for r, col in zip(data[i+1:i+10], columnas): # No tomamos en cuenta el año
        print('{:>5d} \t {}'.format(int(r.contents[0]), col))

### Incorporando la información al DataFrame

Ahora si estamos listos para incrustar la información en cada posición del DataFrame.

Usaremos la función `loc` con la cual podemos indicar el índice del renglón y la columna:

In [ ]:
for i in range(0,340,10):
    for r, col in zip(data[i+1:i+10], columnas):
        sismicidad.loc[data[i].contents[0], col] = int(r.contents[0])

Nuestro DataFrame es el siguiente:

In [ ]:
sismicidad

# Visualización de los datos 

In [ ]:
sismicidad.plot(kind='bar', y = 'TOTAL DE SISMOS')

In [ ]:
sismicidad.columns

In [ ]:
sismicidad.plot.area(alpha=0.5)

In [ ]:
sismicidad.loc['2001'].plot()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sismicidad.loc['2001'][1:].plot()
plt.xticks(rotation=45)
plt.show()

In [ ]:
sismicidad.loc['2001'][1:].plot(kind = 'bar')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sismicidad.plot(subplots=True)

In [ ]:
sismicidad.plot(subplots=True, kind='bar')

In [ ]:
sismicidad.iloc[:,2:5].plot(subplots=True, kind='bar', sharey=True)

In [ ]:
sismicidad.iloc[:,7:9].plot(subplots=True, kind='bar', sharey=True)